https://www.kaggle.com/c/cs5785-fall19-final/

# Create an Ensemble Model

#### Get the Descriptions and Tags from File

In [13]:
import os
import numpy as np
import gensim
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [2]:
def parse_descriptions(data_dir, num_doc):
    docs = []
    for i in range(num_doc):
        path = os.path.join(data_dir, "%d.txt" % i)
        with open(path) as f:
            docs.append(f.read())
    return docs

In [3]:
train_tags = parse_descriptions("cs5785-fall19-final/tags_train", num_doc=10000)
test_tags = parse_descriptions("cs5785-fall19-final/tags_test", num_doc=2000)

In [4]:
def parse_tags(tags):
    result = []
    for doc in tags:
        doc = doc.strip('\n').split('\n')
        cat_it = []
        if doc[0] == '':
            result.append('no tag')
        else:
            for tag in doc:
                split_tag = tag.split(':')
                cat_it.append(split_tag[0])
                cat_it.append(split_tag[1])
            parsed = (' ').join(list(cat_it))
            result.append(parsed)
    return np.array(result)

In [5]:
train_tags_parsed = parse_tags(train_tags)
test_tags_parsed = parse_tags(test_tags)

In [6]:
# function to preprocess data
def preprocessing(data):
    stop_words = set(stopwords.words('english')) # find stop words in English language
    lemmatizer = WordNetLemmatizer() # declare nltk lemmatizer

    # iterate through every sentence and replace it by itself lemmatized, without punctuation and without stop words
    for i in range(len(data)):
        sentence_no_punct = ''
        # remove punctuation
        
        for char in data[i]:
            if char not in string.punctuation:
                sentence_no_punct = sentence_no_punct + char
        data[i] = sentence_no_punct

        word_tokens = word_tokenize(data[i])
    
        # remove stop words and lemmatize
        word_tokens = [lemmatizer.lemmatize(word) for word in word_tokens if word not in stop_words and len(word) > 1]
        word_tokens = [lemmatizer.lemmatize(word, 'v') for word in word_tokens]
        word_tokens = [lemmatizer.lemmatize(word, 'a') for word in word_tokens]
        
        # remove conjunction words
        word_tokens = [word for word in word_tokens if word[-2:] != 'nt']
        (data[i]) = ' '.join(word_tokens)
        
    return data

In [7]:
train_descs = parse_descriptions('cs5785-fall19-final/descriptions_train', 10000)
test_descs  = parse_descriptions('cs5785-fall19-final/descriptions_test', 2000)
train_descs = preprocessing(train_descs)
test_descs  = preprocessing(test_descs)

#### Create the Bag of Words

In [8]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format("../GoogleNews-vectors-negative300.bin.gz", binary=True)

In [9]:
def doc_to_vec(sentence, word2vec):
    # get list of word vectors in sentence
    word_vecs = [word2vec.get_vector(w) for w in sentence.split() if w in word2vec.vocab]
    # return average
    return np.stack(word_vecs).mean(0)

In [10]:
x_train = np.array([doc_to_vec(train_descs[i], word2vec) for i in range(len(train_descs))])
x_test = np.array([doc_to_vec(d, word2vec) for d in test_descs])

#### Get Image Feature Vectors

In [15]:
train_feat_sort = pd.read_csv("sorted_train_features.csv", header = None, index_col=None)
test_feat_sort = pd.read_csv("sorted_test_features.csv", header = None, index_col=None)

scalar = StandardScaler()

train_feat_sort_norm = scalar.fit_transform(train_feat_sort)
test_feat_sort_norm = scalar.fit_transform(test_feat_sort)

#train_inter_feat = pd.read_csv("cs5785-fall19-final/features_train/features_resnet1000intermediate_train.csv", header = None, index_col=None)
#test_inter_feat = pd.read_csv("cs5785-fall19-final/features_test/features_resnet1000intermediate_test.csv", header = None, index_col=None)

In [12]:
#train_feat_2 = train_feat

#for i in range(1000):
    #train_feat[0][i] = train_feat[0][i].replace("images_train/", "").replace(".jpg", "")
    #train_feat[0][i] = temp

#train_feat_sort = train_feat.sort_values(by=0)

#for i in range(len(test_feat[0])):
 #   test_feat[0][i] = int(test_feat[0][i].replace("images_test/", "").replace(".jpg", ""))

#test_feat_sort = test_feat.sort_values(by=0)

In [13]:
#train_feat_sort = train_feat.sort_values(by=0)

In [14]:
#for i in range(len(test_feat[0])):
  #  test_feat[0][i] = int(test_feat[0][i].replace("images_test/", "").replace(".jpg", ""))

In [15]:
#y_train = train_feat_sort.drop(columns=0).to_numpy()
#y_test = test_feat_sort.drop(columns=0).to_numpy()

In [16]:
y_train = train_feat_sort_norm
#.to_numpy()
y_test = test_feat_sort_norm
#.to_numpy()
#y_train_inter = train_inter_feat_sort.drop(columns=0).to_numpy()
#y_test_inter = test_inter_feat_sort.drop(columns=0).to_numpy()

#### Create A Train/Test Split

In [17]:
import random
num_train = 8000
num_dev = 2000
num_test = 2000
split = list(range(num_train + num_dev))
random.shuffle(split)

In [18]:
x_train_dev = x_train[split[:num_train]]
y_train_dev = y_train[split[:num_train]]
x_test_dev = x_train[split[num_train:]]
y_test_dev = y_train[split[num_train:]]

train_tags_dev = train_tags_parsed[split[:num_train]]
test_tags_dev = train_tags_parsed[split[num_train:]]

#### For Later when we need the tags of the whole train and test set

In [19]:
tag_train = np.array([doc_to_vec(d, word2vec) for d in train_tags_parsed])
tag_test = np.array([doc_to_vec(d, word2vec) for d in test_tags_parsed])

#### For Development Right Now when we need the smaller train and test set to check accuracy

In [20]:
train_tags_dev = np.array([doc_to_vec(d, word2vec) for d in train_tags_dev])
test_tags_dev = np.array([doc_to_vec(d, word2vec) for d in test_tags_dev])

### Relate Descriptions and Tags with a Model --- SVM

In [60]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
supvm = MultiOutputRegressor(SVR(gamma = 'scale')).fit(x_train_dev, train_tags_dev)
tag_preds = supvm.predict(x_test_dev)

### Relate Image Features and Descriptions Model

In [21]:
from sklearn.neural_network import MLPRegressor

clf = MLPRegressor(solver='sgd', alpha=1e-5, hidden_layer_sizes=(300,), random_state=1, max_iter = 1000)
clf.fit(x_train_dev, y_train_dev)
image_preds = clf.predict(x_test_dev)

### Lasso Regressor with Cross-Validation to relate descriptors and tags (maybe try instead of mlp too?)

In [22]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

lasso = Lasso()
parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv = 5)
lasso_regressor.fit(x_train_dev, train_tags_dev)
tag_preds = lasso_regressor.predict(x_test_dev)

print(lasso_regressor.best_params_)
print(lasso_regressor.best_score_)

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.023286819458008, tolerance: 0.007494863588362932
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12.949422836303711, tolerance: 0.009234962053596973
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.892945766448975, tolerance: 0.006736687384545803
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dua

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.959437847137451, tolerance: 0.003619154915213585
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.23145866394043, tolerance: 0.004599445499479771
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.647982597351074, tolerance: 0.006147872190922499
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duali

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.615821838378906, tolerance: 0.005801154766231775
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.866389751434326, tolerance: 0.0039514293894171715
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.604494094848633, tolerance: 0.0032058265060186386
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Du

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.298031330108643, tolerance: 0.002721499651670456
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.420781135559082, tolerance: 0.003281639888882637
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.582625389099121, tolerance: 0.004343536216765642
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dual

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.965215682983398, tolerance: 0.003755714278668165
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.93845272064209, tolerance: 0.003551977453753352
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.768080711364746, tolerance: 0.005819080397486687
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duali

...

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.921438217163086, tolerance: 0.005785527639091015
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.645620346069336, tolerance: 0.010319039225578308
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.2683348655700684, tolerance: 0.003089922247454524
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Dua

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.1220998764038086, tolerance: 0.005293581169098616
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.8435797691345215, tolerance: 0.004680251237004995
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.5572595596313477, tolerance: 0.006683570798486471
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. D

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.7418980598449707, tolerance: 0.004079222213476896
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.0089335441589355, tolerance: 0.002661770675331354
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.1088323593139648, tolerance: 0.0036148028448224068
  positive)
C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. 

{'alpha': 1e-08}
-0.0024795431643724443


C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.3382439613342285, tolerance: 0.007789814379066229
  positive)


Best lasso params: {'alpha': 1e-08}
-0.0024774808902293445

### Ridge Regressor with Cross-Validation 

In [42]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
ridge = Ridge()
parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]}

ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv = 5)
ridge_regressor.fit(x_train_dev, train_tags_dev)
tag_preds = ridge_regressor.predict(x_test_dev)

print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

{'alpha': 1}
-0.0024293445982038974


In [24]:
print(tag_preds.shape)
print(test_tags_dev.shape)
print(y_train_dev.shape)
print(x_train_dev.shape)

(2000, 300)
(2000, 300)
(8000, 1000)
(8000, 300)


Best Ridge parameters: {'alpha': 1}
-0.0024467481765896083

### Combine the Models

#### Try Evaluating the Nearest Neighbors and Take A New Ranking Based on Both

In [34]:
# function to return the number of nearest neighbors specified
def knn_function(data, point, neighbors_number):
    knn = NearestNeighbors(n_neighbors=neighbors_number)
    knn.fit(data)
    return knn.kneighbors(point, neighbors_number, return_distance=False)

In [35]:
tag_neighbors = knn_function(test_tags_dev, tag_preds, 50)
image_neighbors = knn_function(y_test_dev, image_preds, 50)

In [36]:
neighbors1 = []
for i in range(len(tag_neighbors)):
    neighbors1.append(np.array(list(set(tag_neighbors[i]) & set(image_neighbors[i]))[:20]))

In [37]:
neighbors1 = np.array(neighbors1)

### Evaluate the Model

In [38]:
def scores(i):
    return (20+1-(i+1))/20

In [39]:
def eval_accuracy(ytest, neighbors):
    # EVALUATE THE MODEL USING THE MEAN AVERAGE PRECISION AT 20
    scs = []
    for i in range(len(neighbors)):
        good = False
        for j, n in enumerate(neighbors[i]):
            if i == n:
                scs.append(scores(j))
                good = True
                break
        if good == False:
            scs.append(0)
    return sum(scs)/len(neighbors)

In [40]:
#print('accuracy of svm tags', eval_accuracy(y_test_dev, tag_neighbors[:20]))

In [41]:
print('accuracy of lasso tags', eval_accuracy(y_test_dev, tag_neighbors[:20]))

accuracy of ridge tags 0.20500000000000002


In [42]:
print('accuracy of image features', eval_accuracy(y_test_dev, image_neighbors[:20]))

accuracy of image features 0.2375


In [43]:
#print('accuracy of svm tags combined with image features', eval_accuracy(y_test_dev, neighbors1))

In [44]:
print('accuracy of ridge tags combined with image features', eval_accuracy(y_test_dev, neighbors1))

accuracy of ridge tags combined with image features 0.3320000000000003
